In [ ]:
# AB Testi ile Bidding Yöntemlerinin Dönüşümünün Karşılaştırılması

# İş Problemi

"""
- Facebook kısa süre önce mevcut "maximum bidding" adı verilen 
teklif verme türüne alternatif olarak yeni bir teklif türü olan 
"average bidding"’i tanıttı. 

- Müşterilerimizden biri olan bombabomba.com, bu yeni özelliği test 
etmeye karar verdi ve average bidding'in maximum bidding'den
daha fazla dönüşüm getirip getirmediğini anlamak için bir A/B 
testi yapmak istiyor.
 
- A/B testi 1 aydır devam ediyor ve bombabomba.com şimdi sizden 
bu A/B testinin sonuçlarını analiz etmenizi bekliyor.
Bombabomba.com için nihai başarı ölçütü Purchase'dır. Bu 
nedenle, istatistiksel testler için Purchase metriğine 
odaklanılmalıdır.
"""

In [ ]:
# Veri Seti Hikayesi

"""Bir firmanın web site bilgilerini içeren bu veri setinde kullanıcıların gördükleri ve tıkladıkları reklam sayıları gibi bilgilerin yanı sıra 
buradan gelen kazanç bilgileri yer almaktadır. Kontrol ve Test grubu olmak üzere iki ayrı veri seti vardır. Bu veri setleri
ab_testing.xlsx excel’inin ayrı sayfalarında yer almaktadır. Kontrol grubuna Maximum Bidding, test grubuna Average
Bidding uygulanmıştır"""

# Variables:

# Impression: Reklam görüntüleme sayısı
# Click: Görüntülenen reklama tıklama sayısı
# Purchase: Tıklanan reklamlar sonrası satın alınan ürün sayısı
# Earning: Satın alınan ürünler sonrası elde edilen kazanç

In [73]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# !pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [74]:
# Görev 1: Veriyi Hazırlama ve Analiz Etme

# Adım 1: ab_testing_data.xlsx adlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız

df_control = pd.read_excel("C:/Users/merve/Documents/AB/datasets/week6/ab_testing.xlsx", sheet_name="Control Group")
df_control.head()

,Impression,Click,Purchase,Earning
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


In [75]:
df_test = pd.read_excel("C:/Users/merve/Documents/AB/datasets/week6/ab_testing.xlsx", sheet_name="Test Group")
df_test.head()

,Impression,Click,Purchase,Earning
0,120103.50380,3216.54796,702.16035,1939.61124
1,134775.94336,3635.08242,834.05429,2929.40582
2,107806.62079,3057.14356,422.93426,2526.24488
3,116445.27553,4650.47391,429.03353,2281.42857
4,145082.51684,5201.38772,749.86044,2781.69752


In [76]:
# Adım 2: Kontrol ve test grubu verilerini analiz ediniz

def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head())
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Describe #####################")
    print(dataframe.describe().T)
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

In [77]:
check_df(df_control)

##################### Shape #####################
(40, 4)
##################### Types #####################
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
##################### Head #####################
    Impression      Click  Purchase    Earning
0  82529.45927 6090.07732 665.21125 2311.27714
1  98050.45193 3382.86179 315.08489 1742.80686
2  82696.02355 4167.96575 458.08374 1797.82745
3 109914.40040 4910.88224 487.09077 1696.22918
4 108457.76263 5987.65581 441.03405 1543.72018
##################### NA #####################
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64
##################### Describe #####################
              count         mean         std         min         25%  \
Impression 40.00000 101711.44907 20302.15786 45475.94296 85726.69035   
Click      40.00000   5100.65737  1329.98550  2189.75316  4124.30413   
Purchase   40.00000    550.89406   134.10820   267.02894   470.095

In [78]:
check_df(df_test)

##################### Shape #####################
(40, 4)
##################### Types #####################
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
##################### Head #####################
    Impression      Click  Purchase    Earning
0 120103.50380 3216.54796 702.16035 1939.61124
1 134775.94336 3635.08242 834.05429 2929.40582
2 107806.62079 3057.14356 422.93426 2526.24488
3 116445.27553 4650.47391 429.03353 2281.42857
4 145082.51684 5201.38772 749.86044 2781.69752
##################### NA #####################
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64
##################### Describe #####################
              count         mean         std         min          25%  \
Impression 40.00000 120512.41176 18807.44871 79033.83492 112691.97077   
Click      40.00000   3967.54976   923.09507  1836.62986   3376.81902   
Purchase   40.00000    582.10610   161.15251   311.62952    444

In [79]:
# Adım 3: Analiz işleminden sonra concat metodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

# 1. yol

df_control["group"] = "control"
df_test["group"] = "test"

df = pd.concat([df_control, df_test], axis=0, ignore_index=True) 
df

,Impression,Click,Purchase,Earning,group
0,82529.45927,6090.07732,665.21125,2311.27714,control
1,98050.45193,3382.86179,315.08489,1742.80686,control
2,82696.02355,4167.96575,458.08374,1797.82745,control
3,109914.40040,4910.88224,487.09077,1696.22918,control
4,108457.76263,5987.65581,441.03405,1543.72018,control
...,...,...,...,...,...
75,79234.91193,6002.21358,382.04712,2277.86398,test
76,130702.23941,3626.32007,449.82459,2530.84133,test
77,116481.87337,4702.78247,472.45373,2597.91763,test
78,79033.83492,4495.42818,425.35910,2595.85788,test


In [63]:
# 2. yol

df2 = pd.concat([df_control, df_test], join="inner", ignore_index=False)
df2

,Impression,Click,Purchase,Earning,group
0,82529.45927,6090.07732,665.21125,2311.27714,control
1,98050.45193,3382.86179,315.08489,1742.80686,control
2,82696.02355,4167.96575,458.08374,1797.82745,control
3,109914.40040,4910.88224,487.09077,1696.22918,control
4,108457.76263,5987.65581,441.03405,1543.72018,control
...,...,...,...,...,...
35,79234.91193,6002.21358,382.04712,2277.86398,test
36,130702.23941,3626.32007,449.82459,2530.84133,test
37,116481.87337,4702.78247,472.45373,2597.91763,test
38,79033.83492,4495.42818,425.35910,2595.85788,test


In [ ]:
# Görev 2: A/B Testinin Hipotezinin Tanımlanması

# Adım 1: Hipotezi tanımlayınız.

# Maximum Bidding ve Average Bidding teklif türlerinin getirdiği dönüşüm yani purchase ortalamaları arasında bir fark yoktur

# H0 : M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.)
# H1 : M1!= M2 (Kontrol grubu ve test grubu satın alma ortalamalarıarasında fark vardır.)

In [80]:
# Adım 2: Kontrol ve test grubu için purchase (kazanç) ortalamalarını analiz ediniz.

# 1. yol

df.groupby("group").agg({"Purchase": "mean"})

# Yorum: kontrol grubu purchase ortalaması test grubundan daha düşük görünüyor 
# Bu iki grubun ortalama farklılığının istatistiki olarak anlamlı olup olmadığını araştırmalıyız

,Purchase
group,
control,550.89406
test,582.10610


In [81]:
# 2. yol

df_test["Purchase"].mean() #582.10
df_control["Purchase"].mean() #550.89

550.8940587702316

In [82]:
# Görev 3: Hipotez Testinin Gerçekleştirilmesi

# Adım 1: Hipotez testi yapılmadan önce varsayım kontrollerini yapınız. Bunlar Normallik Varsayımı 
# ve Varyans Homojenliğidir. Kontrol ve test grubunun normallik varsayımına uyup uymadığını Purchase değişkeni 
# üzerinden ayrı ayrı test ediniz.

"""Normallik Varsayımı :
H0: Normal dağılım varsayımı sağlanmaktadır. 
H1: Normal dağılım varsayımı sağlanmamaktadır.
p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
Test sonucuna göre normallik varsayımı kontrol ve test grupları için sağlanıyor mu ? 
Elde edilen p-value değerlerini yorumlayınız."""

test_stat, pvalue = shapiro(df.loc[df["group"] == "control", "Purchase"]) # shapiro önce ilgili grubu ister (sigara içen yes grubu)
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# p-value > 0.05 null hyp reddedilemez yani normallik varsayımı sağlanıyor

Test Stat = 0.9773, p-value = 0.5891


In [83]:
test_stat, pvalue = shapiro(df.loc[df["group"] == "test", "Purchase"]) # shapiro önce ilgili grubu ister (sigara içen yes grubu)
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
# p-value > 0.05 null hyp reddedilemez yani normallik varsayımı sağlanıyor kontrol grubu için

Test Stat = 0.9589, p-value = 0.1541


In [84]:
# 2. yol

test_stat, pvalue = shapiro(df_control["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

test_stat, pvalue = shapiro(df_test["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891
Test Stat = 0.9589, p-value = 0.1541


In [85]:
"""
Varyans: verilerin aritmetik ortalamadan sapmalarının karelerinin toplamı (std'nin köksüz hali)
Varyans Homojenliği :
H0: Varyanslar homojendir.
H1: Varyanslar homojen Değildir.
p < 0.05 H0 RED , p > 0.05 H0 REDDEDİLEMEZ
Kontrol ve test grubu için varyans homojenliğinin sağlanıp sağlanmadığını 
Purchase değişkeni üzerinden test ediniz. 
Test sonucuna göre normallik varsayımı sağlanıyor mu? 
Elde edilen p-value değerlerini yorumlayınız. """

# 1. yol

test_stat, pvalue = levene(df.loc[df["group"] == "control", "Purchase"],
                           df.loc[df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# p-value > 0.05 null hyp reddedilemez, varyanslar homojendir

Test Stat = 2.6393, p-value = 0.1083


In [86]:
# 2. yol

test_stat, pvalue = levene(df_test["Purchase"],
                           df_control["Purchase"].dropna()) # dropna eksik değer varse levene testte sorun çıkar

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [ ]:
# Adım 2: Normallik Varsayımı ve Varyans Homojenliği sonuçlarına göre uygun testi seçiniz

# 2 varsayım da sağlandığı için bağımsız iki örneklem t testi (parametrik test) uygulayabiliriz

In [87]:
# Adım 3: Test sonucunda elde edilen p_value değerini göz önünde bulundurarak 
# kontrol ve test grubu satın alma ortalamaları arasında istatistiki 
# olarak anlamlı bir fark olup olmadığını yorumlayınız

# 1. yol

test_stat, pvalue = ttest_ind(df.loc[df["group"] == "control", "Purchase"],
                              df.loc[df["group"] == "test", "Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

# p-value > 0.05 null hyp reddedilemez, Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur
# Maximum Bidding ve Average Bidding teklif türlerinin getirdiği dönüşüm yani purchase ortalamaları arasında istatistiki olarak anlamlı bir fark yoktur

Test Stat = -0.9416, p-value = 0.3493


In [ ]:
# H0 : M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.)
# H1 : M1!= M2 (Kontrol grubu ve test grubu satın alma ortalamalarıarasında fark vardır.)

In [88]:
# 2. yol

test_stat, pvalue = ttest_ind(df_test["Purchase"],
                              df_control["Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9416, p-value = 0.3493


In [103]:
# Tutku'nun fonksiyonu

def AB_Test(dataframe, group, target):

    # Necessary packages
    from scipy.stats import shapiro
    import scipy.stats as stats

    # # Split A/B
    control = dataframe[dataframe[group] == "control"][target] #Old Design
    test = dataframe[dataframe[group] == "test"][target] #New Desing

    # Assumption of the Normality 
    normality_control = shapiro(control)[1] < 0.05
    normality_test = shapiro(test)[1] < 0.05

    # H0: Data follow a normal distribution.- False
    # H1: Data do not follow a normal distribution. - True

    if (normality_control == False) & (normality_test == False):  # "H0: Data follow a normal distribution
        # Parametric Test
        # Assumption: Homogeneity of variances

        leveneTest = stats.levene(control, test)[1] < 0.05
        # H0: Homogeneity: False
        # H1: Heterogeneous: True

        if leveneTest == False:
            # Homogeneity
            ttest = stats.ttest_ind(control, test, equal_var=True)[1] # Attention! equal_var=True
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
        else:
            # Heterogeneous
            ttest = stats.ttest_ind(control, test, equal_var=False)[1] #Attention! equal_var=False
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
    else:
        # Non-Parametric Test
        ttest = stats.mannwhitneyu(control, test)[1]
        # H0: M1 == M2 - False
        # H1: M1 != M2 - True

    # Result
    temp = pd.DataFrame({
        "AB Hypothesis": [ttest < 0.05],
        "p-value": [ttest]
    })
    temp["Test Type"] = np.where((normality_control == False) & (normality_test == False), "Parametric", "Non-Parametric")
    temp["AB Hypothesis"] = np.where(temp["AB Hypothesis"] == False, "Fail to Reject H0", "Reject H0")
    temp["Comment"] = np.where(temp["AB Hypothesis"] == "Fail to Reject H0", "A/B groups are similar!",
                               "A/B groups are not similar!")

    # Columns
    if (normality_control == False) & (normality_test == False):
        temp["Homogeneity"] = np.where(leveneTest == False, "Yes", "No")
        temp = temp[["Test Type", "Homogeneity", "AB Hypothesis", "p-value", "Comment"]]
    else:
        temp = temp[["Test Type", "AB Hypothesis", "p-value", "Comment"]]

    # Print Hypothesis
    print("# A/B Testing Hypothesis")
    print("H0: A == B")
    print("H1: A != B", "\n")

    return temp

In [104]:
AB_Test(df, "group", "Purchase")

# A/B Testing Hypothesis
H0: A == B
H1: A != B 



,Test Type,Homogeneity,AB Hypothesis,p-value,Comment
0,Parametric,Yes,Fail to Reject H0,0.34933,A/B groups are similar!


In [ ]:
# Görev 4: Sonuçların Analizi

# Adım 1: Hangi testi kullandınız, sebeplerini belirtiniz

# Hem normallik hem de varyans homojenliği varsayımları sağlandığı (ve iki grup arasındaki istatistiksel anlamları ortaya çıkarabilmek) için 
# bağımsız iki örneklem t testi (parametrik test) kullandım

# Diğer süreçler:
# Kontrol ve test grubunun normallik varsayımına uyup uymadığını test etmek için Shapiro testi kullandım
# 2 grupta da normallik varsayımı sağlandığı için (normal dağılıma uyduğu için) varyans homojenliği varsayımını kontrol ettim
# Varyans homojenliği varsayımını test etmek için Levene testi kullandım.
# 2 grupta varyans homojenliği olduğu sonucuna vardım
# Sonuçta kontrol ve test gruplarının ortalamaları arasında fark olduğu görülse bile
# bunun istatistiksel olarak anlamlı olmadığı yapılan analizler sonucu tespit edilmiştir.

In [ ]:
# Adım 2: Elde ettiğiniz test sonuçlarına göre müşteriye tavsiyede bulununuz.

# yeni sistem purchase kısmında anlamlı bir fark yaratmadığı için farklı sistemler denenebilir
# Web sitesine eklenen yeni sistemi tekrardan incelemek için daha fazla örneklem ile yeni bir test yapılabilir. 
# Test ettiğimiz yeni sistem kazanç ortalamalarında anlamlı bir farklılık yaratmamıştır. Şu an için hayata geçirilmesine gerek yoktur.
# Yapılan iş tesadüfen oluşmuştur ve 2. çalışmaya gerek yoktur. (mentorumuzun yorumu)
# Tıklanma, Etkileşim, Kazanç ve Dönüşüm oranlarındaki farklılıklar değerlendirilip istatistikleri 
# incelenebilir ona göre 2 metoddan hangisinin kullanılacağına karar verilebilir
# Eğer 2 metodu birden kullanmanın ciddi maliyeti yoksa data toplanmaya devam edilip 
# Kıyasa devam edilebilir
# Facebookta tıklanma başına para ödendiği için hangi yöntemde tıklanma oranı daha düşük 
# araştırması yapılabilir ve CTR (click through tate tıklama oranı) oranına bakılabilir

In [109]:
round(0.051,2)

0.05

In [110]:
round(0.059,2)

0.06

In [123]:
import numpy as np
arr = [2,3]
np.quantile(arr, 0.25)

2.25

In [56]:
series = pd.Series([1, 2, 3, 4, 5])

In [105]:
series.quantile(0.9) #0.5

3.7

In [124]:
series2 = pd.Series([2, 3])

In [125]:
# 0.25*1
# tam kısmı 0, percentage 0.25
# hiç bir şey arasında değil
series2[0] + 0.25*(series2[1]-series2[0])

2.25

In [58]:
series = pd.Series([1, 2, 3, 4])

In [59]:
series.quantile()

2.5

In [112]:
n = len(series)
n

4

In [117]:
0.9*(n-1) # 2 ve 3. değerler arasında (yani 3 ile 4)
# Tam kısmı 2, ondalık kısmı 0.7

2.7

In [115]:
# Linear Interpolation
quantile = series[2] + 0.7*(series[3] - series[2])
quantile

3.7

In [116]:
4*0.9

3.6

In [106]:

series.quantile([.25, .5, .75])

0.25000   1.75000
0.50000   2.50000
0.75000   3.25000
dtype: float64

In [ ]:
[1, 2, 3, 4]
